In [20]:
#This file creates the classifier to detect fake news
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, average_precision_score, recall_score, f1_score

In [21]:
#load csv of data

##Load Fake Data set and give it a label of 1 
fake_df = pd.read_csv('datasets/Fake.csv')
fake_df['label'] = 1

true_df = pd.read_csv('datasets/True.csv')
true_df['label'] = 0

labeled_df = pd.concat([fake_df, true_df])
labeled_df = labeled_df.sample(frac=1).reset_index(drop=True)
##Save a copy of labled dataset for ease of use next time
#labeled_df.to_csv('datasets/labeled.csv')


In [22]:
#grab labels
labels = labeled_df.label

In [23]:
#Split dataset
x_train, x_test, y_train, y_test = train_test_split(labeled_df['text'], labels, test_size = 0.2, random_state = 7)

In [24]:
#initialize TFIDF
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

In [25]:
#Vectorize test and train set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [26]:
#Create the Classfier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [33]:
#Test Classfier
y_pred = pac.predict(tfidf_test)
accuracy = accuracy_score(y_test,y_pred)
precision = average_precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)
print(f'Accuracy: {round(accuracy*100,2)}%')
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('F1 Score: {}'.format(round(f1,4)))

Accuracy: 99.42%
Precision: 0.9921
Recall: 0.9941
F1 Score: 0.9945


In [34]:
#Create confusion Matrix
#FORMAT:
#   True Pos, False Neg
#   False Pos, True Neg
confusion_matrix(y_test,y_pred, labels=[1, 0])

array([[4693,   28],
       [  24, 4235]])